<p>Su Doku (Japanese meaning <i>number place</i>) is the name given to a popular puzzle concept. Its origin is unclear, but credit must be attributed to Leonhard Euler who invented a similar, and much more difficult, puzzle idea called Latin Squares. The objective of Su Doku puzzles, however, is to replace the blanks (or zeros) in a 9 by 9 grid in such that each row, column, and 3 by 3 box contains each of the digits 1 to 9. Below is an example of a typical starting puzzle grid and its solution grid.</p>
<div class="center">
<img src="https://projecteuler.net/project/images/p096_1.png" alt="" /><br /></div>
<img src="https://projecteuler.net/project/images/p096_2.png" alt="" /><br /></div>

<p>A well constructed Su Doku puzzle has a unique solution and can be solved by logic, although it may be necessary to employ "guess and test" methods in order to eliminate options (there is much contested opinion over this). The complexity of the search determines the difficulty of the puzzle; the example above is considered <i>easy</i> because it can be solved by straight forward direct deduction.</p>
<p>The 6K text file, <a href="https://projecteuler.net/project/resources/p096_sudoku.txt">sudoku.txt</a> (right click and 'Save Link/Target As...'), contains fifty different Su Doku puzzles ranging in difficulty, but all with unique solutions (the first puzzle in the file is the example above).</p>
<p>By solving all fifty puzzles find the sum of the 3-digit numbers found in the top left corner of each solution grid; for example, 483 is the 3-digit number found in the top left corner of the solution grid above.</p>

**Solution(s):**
We parse through the text document, converting the text into a list of 50 $9 \times 9$ matrices that represent the Su Doku puzzles. To solve each individual puzzle, we set the problem up in a manner where we're solving for an [exact cover](https://en.wikipedia.org/wiki/Exact_cover#Sudoku). We use the PyPi algorithm to help us find the exact cover. Once we have the puzzle's solution, we add the 3-digit number from the first row to a running total.

In [ ]:
import numpy as np
import sys
import exact_cover as ec
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Here we go through the list of puzzles and save them as a list of strings.

puzzleFile = open("p096_sudoku.txt", "r")               # read the file
puzzles = []                                     # initialize a list to store the strings
counter = 0
puzzle = []
while True:
     # Get next line from file
    row = puzzleFile.readline()
  
    # if line is empty
    # end of file is reached
    if not row:
        break
    if (counter % 10):
        puzzle.append(row[:-1])
    else:
        puzzles.append(puzzle)
        puzzle = []
    counter += 1
puzzles.append(puzzle)

puzzleFile.close()
puzzles = puzzles[1:]

In [ ]:
# Convert the list of strings into a list of matrices
matrices = []
for puzzle in puzzles:
    matrix = []
    for line in puzzle:
        row = [int(a) for a in list(line)]
        matrix.append(row)
    matrices.append(matrix)

In [ ]:
# Fix the last matrix, since the last element got cut off
matrices[-1][-1].append(6)

In [ ]:
# Now we set the problem up as an exact cover, solve it, and add the 3 digit number to our tot
tot = 0
for n in range(50):
    print(n, tot)
    matrix = matrices[n]
    S = np.zeros([729,4*81])            # The matrix described in the Wikipedia page, capturing our constraints
    for i in range(81):
        S[9*i:9*i + 9, i] = np.ones([9])        # The first constraint
        row = i//9
        col = i%9
        box = 3*(row//3) + (col//3)
        elt = matrix[row][col]
        if elt:                                 # If the square is filled in, we fill in the corresponding constraints
            S[81*row+9*col+elt-1][81+9*row+elt-1] = 1
            S[81*row+9*col+elt-1][2*81+9*col+elt-1] = 1
            S[81*row+9*col+elt-1][3*81+9*box+elt-1] = 1
        else:
            # Here we don't know the square's value, so we fill in the possible values
            for j in range(9):

                S[(81*row+9*col+j),81+9*row + j] = 1
                S[(81*row+9*col+j),2*81+9*col + j] = 1
                S[(81*row+9*col+j),3*81+9*box + j] = 1

    solution = ec.get_exact_cover(S)

    # Now convert the rows from the exact cover back to a matrix
    solMat = np.zeros([9,9])
    for i in solution:
        elt = (i % 9)+1
        row = i//81
        col = (i-81*row)//9
        solMat[row][col] = elt
        
    # Finally, add the three digit number to our running total.
    tot += 100*solMat[0][0]+10*solMat[0][1] + solMat[0][2]
tot